# Introduction to Perceval and Remote Quantum Computing with the Quandela Cloud

In this tutorial, we will cover some basic code to get our hands on Perceval.


## I. Introduction

### 1. Perceval installation

In [ ]:
import perceval as pcvl
pcvl.__version__

In [ ]:
from perceval.components.unitary_components import PS, BS, PERM
from perceval.algorithm import Sampler, Analyzer
import numpy as np

### 2. BasicStates

In Linear Optical (LO) circuits, photons can have many discrete degrees of freedom, which can be used as computational modes: frequency, polarisation, position, ...

We represent these degrees of freedom with Fock states. If we have $n$ photons over $m$ modes, the Fock state $|s_1,s_2,...,s_m\rangle$ means we have $s_i$ photons in the $i^{th}$ mode, i.e. $\sum_{i=1}^m s_i =n$.

In Perceval, we will use the module `pcvl.BasicState`

In [ ]:
## Difference syntaxes of BasicStates (list, string, etc.)
bs1 = pcvl.BasicState('|0,2,0,1>')
bs2 = pcvl.BasicState([0, 2, 0, 1])

if bs1==bs2:print('Those are the same states')

## You can iterate on modes
for i, photon_count in enumerate(bs2):
    print(f'There are {photon_count} photons in mode {i}')

### 3. LO-components

LO components are the elementary blocks which will act on our Fock states.

It's important to know all the possible components that can be found in Perceval and understand their effects.

In [ ]:
## Permutation
perm=PERM([2,0,1])

print(perm.name)
print(perm.describe())
pcvl.pdisplay(perm.definition())
pcvl.pdisplay(perm)

The output matrix from above is $\left[\begin{matrix}0 & 1 & 0\\0 & 0 & 1\\1 & 0 & 0\end{matrix}\right]$.

In [ ]:
## Phase shifter
ps = PS(phi=np.pi)

print(ps.name)
print(ps.describe())  
pcvl.pdisplay(ps.definition()) 
pcvl.pdisplay(ps)  # A pdisplay call on a circuit/processor needs to be the last line of a cell

The output matrix from above is $\left[\begin{matrix}e^{i \phi}\end{matrix}\right]$.

In [ ]:
## Beam splitters
bs_rx = BS.Rx()  # By default, a beam splitter follows the Rx gate convention, so bs=BS() has the same matrix

# But other conventions of beam splitter phase exist too:
bs_h = BS.H() 
bs_ry = BS.Ry()

## Check the difference in the unitary definition:
print('BS.Rx() unitary matrix')
pcvl.pdisplay(bs_rx.definition())
print('BS.H() unitary matrix')
pcvl.pdisplay(bs_h.definition())
print('BS.Ry() unitary matrix')
pcvl.pdisplay(bs_ry.definition())
print('BS displays its convention as a small label:')
pcvl.pdisplay(bs_ry)

The output matrices from above are:

$\left[\begin{matrix}e^{i \left(\phi_{tl} + \phi_{tr}\right)} \cos{\left(\frac{\theta}{2} \right)} & i e^{i \left(\phi_{bl} + \phi_{tr}\right)} \sin{\left(\frac{\theta}{2} \right)}\\i e^{i \left(\phi_{br} + \phi_{tl}\right)} \sin{\left(\frac{\theta}{2} \right)} & e^{i \left(\phi_{bl} + \phi_{br}\right)} \cos{\left(\frac{\theta}{2} \right)}\end{matrix}\right]$

$\left[\begin{matrix}e^{i \left(\phi_{tl} + \phi_{tr}\right)} \cos{\left(\frac{\theta}{2} \right)} & e^{i \left(\phi_{bl} + \phi_{tr}\right)} \sin{\left(\frac{\theta}{2} \right)}\\e^{i \left(\phi_{br} + \phi_{tl}\right)} \sin{\left(\frac{\theta}{2} \right)} & - e^{i \left(\phi_{bl} + \phi_{br}\right)} \cos{\left(\frac{\theta}{2} \right)}\end{matrix}\right]$

$\left[\begin{matrix}e^{i \left(\phi_{tl} + \phi_{tr}\right)} \cos{\left(\frac{\theta}{2} \right)} & - e^{i \left(\phi_{bl} + \phi_{tr}\right)} \sin{\left(\frac{\theta}{2} \right)}\\e^{i \left(\phi_{br} + \phi_{tl}\right)} \sin{\left(\frac{\theta}{2} \right)} & e^{i \left(\phi_{bl} + \phi_{br}\right)} \cos{\left(\frac{\theta}{2} \right)}\end{matrix}\right]$

In [ ]:
# You can ask for the symbolic matrix value of your component with the attribute U
my_ps = PS(phi=np.pi/8)
pcvl.pdisplay(my_ps.U)
# And for the numerical value with the method compute_unitary
pcvl.pdisplay(my_ps.compute_unitary())
print('')

# If you do it for a beam splitter, you can see that by default theta=pi/2, and the phi's are 0
print('A default beam splitter:')
pcvl.pdisplay(BS().compute_unitary())  # this is a balanced beam splitter
print('')

# To control the value of the parameters of a component, several choices are possible:
#  - by setting a numerical value during the creation of the component
print('A beam splitter with a numerical value for theta:')
bs_rx = BS.Rx(theta=10)
pcvl.pdisplay(bs_rx.U)
pcvl.pdisplay(bs_rx.compute_unitary())
print('')

#  - by using the syntax pcvl.P to create a symbolic variable 
#    (note that you cannot compute the numerical value of your component anymore)
print('A phase shifter with a symbolic value for phi:')
ps = PS(phi=pcvl.P('\psi'))
pcvl.pdisplay(ps.U)
print('')

#  - you can still modify the value of a symbolic variable after its creation
#    This is not true for a numerical variable!
print('A beam splitter with a symbolic variable...')
bs_rx = BS(theta=pcvl.P('toto'))
pcvl.pdisplay(bs_rx.U)
bs_rx.assign({'toto':10})
print('... set to a numerical value')
pcvl.pdisplay(bs_rx.compute_unitary())
print('')

# To check which parameters can be modified, you can call the method get_parameters
# You can also directly change the output of get_parameters to change the values of the parameters
bs_rx = BS(theta=pcvl.P('toto'), phi_tl = pcvl.P('tata'), phi_tr = -1)
parameters = bs_rx.get_parameters()
parameters[0].set_value(np.pi)
print('Modified parameters...')
for param in parameters:
    print('    ', param)
print('... and successfully modified beam splitter:')
pcvl.pdisplay(bs_rx.U)

_The above output in markdown:_

$\left[\begin{matrix}e^{0.392699081698724 i}\end{matrix}\right]$

$\left[\begin{matrix}0.92388 + 0.382683 i\end{matrix}\right]$

A default beam splitter:

$\left[\begin{matrix}\frac{\sqrt{2}}{2} & \frac{\sqrt{2} i}{2}\\\frac{\sqrt{2} i}{2} & \frac{\sqrt{2}}{2}\end{matrix}\right]$

A beam splitter with a numerical value for theta:

$\left[\begin{matrix}\cos{\left(5 \right)} & i \sin{\left(5 \right)}\\i \sin{\left(5 \right)} & \cos{\left(5 \right)}\end{matrix}\right]$

$\left[\begin{matrix}0.283662 & - 0.958924 i\\- 0.958924 i & 0.283662\end{matrix}\right]$

A phase shifter with a symbolic value for phi:

$\left[\begin{matrix}e^{i \psi}\end{matrix}\right]$

A beam splitter with a symbolic variable...

$\left[\begin{matrix}\cos{\left(\frac{toto}{2} \right)} & i \sin{\left(\frac{toto}{2} \right)}\\i \sin{\left(\frac{toto}{2} \right)} & \cos{\left(\frac{toto}{2} \right)}\end{matrix}\right]$

... set to a numerical value

$\left[\begin{matrix}0.283662 & - 0.958924 i\\- 0.958924 i & 0.283662\end{matrix}\right]$

Modified parameters...

     Parameter(name='toto', value=3.141592653589793, min_v=0.0, max_v=12.566370614359172)

     Parameter(name='tata', value=None, min_v=0.0, max_v=6.283185307179586)

... and successfully modified beam splitter:

$\left[\begin{matrix}6.12323399573677 \cdot 10^{-17} e^{i \left(tata + 5.28318530717959\right)} & 1.0 i e^{5.28318530717959 i}\\1.0 i e^{i tata} & 6.12323399573677 \cdot 10^{-17}\end{matrix}\right]$

In [ ]:
## to understand the conventions, you can note that a BS.Rx with the 4 phases phi (top left/right and bottom left/right) can be represented like that 
bs_rx_circuit=pcvl.Circuit(2) // (0,PS(phi=pcvl.P('phi_tl'))) // (1,PS(phi=pcvl.P('phi_bl'))) // BS(theta=pcvl.P('theta')) // (0,PS(phi=pcvl.P('phi_tr'))) // (1,PS(phi=pcvl.P('phi_br')))

pcvl.pdisplay(bs_rx_circuit.U)

# we can check it's the same as bs_rx.definition()
pcvl.pdisplay(bs_rx_circuit)

## For this cell, we needed the syntax to builds circuits... Good transition!

The output matrix from above is $\left[\begin{matrix}e^{i \left(\phi_{tl} + \phi_{tr}\right)} \cos{\left(\frac{\theta}{2} \right)} & i e^{i \left(\phi_{bl} + \phi_{tr}\right)} \sin{\left(\frac{\theta}{2} \right)}\\i e^{i \left(\phi_{br} + \phi_{tl}\right)} \sin{\left(\frac{\theta}{2} \right)} & e^{i \left(\phi_{bl} + \phi_{br}\right)} \cos{\left(\frac{\theta}{2} \right)}\end{matrix}\right]$.

## II. LO-circuits

From the LO-components, we can build a LO-circuit, i.e. a sequence of those components acting on our different modes.

### 1. Syntax

In [ ]:
circuit = pcvl.Circuit(3)  # Create a 3-mode circuit

circuit.add(0, BS())
circuit.add(0, PS(phi=np.pi/2)).add(1, PS(phi=pcvl.P('phi'))).add(1, BS())

# Equivalent syntax:
# circuit // BS() // PS(phi=np.pi/2) // (1, PS(phi=pcvl.P('phi'))) // (1, BS())

pcvl.pdisplay(circuit.U)
pcvl.pdisplay(circuit)

The output matrix from above is $\left[\begin{matrix}\frac{\sqrt{2} e^{1.5707963267949 i}}{2} & \frac{\sqrt{2} i e^{1.5707963267949 i}}{2} & 0\\\frac{i e^{i \phi}}{2} & \frac{e^{i \phi}}{2} & \frac{\sqrt{2} i}{2}\\- \frac{e^{i \phi}}{2} & \frac{i e^{i \phi}}{2} & \frac{\sqrt{2}}{2}\end{matrix}\right]$.

The syntax ``pcvl.P('phi')`` allows you to use parameters in the circuit, where you can assign a value or not. The behavior of the parameters of a circuit is similar to the case of the components.

For instance, you can use :

In [ ]:
params=circuit.get_parameters()
print(params) # list of the parameters

# the value is None, but we can change that with
params[0].set_value(np.pi)
pcvl.pdisplay(circuit)

### 2. Mach-Zehnder interferometers

The beamsplitter's angle $\theta$ can also be defined as a parameter.

However, as the reflexivity depends on the mirror, it's hard to have adaptibility on the angle. 
Therefore, in practice, we use a [Mach-Zehnder Interferometer](https://en.wikipedia.org/wiki/Mach%E2%80%93Zehnder_interferometer). 

The beamsplitter with a parameterised $\theta$ is therefore implemented with a parameterised phase shifter $\phi$ between two fixed beamsplitters.




In [ ]:
## TODO: build a circuit implementing an MZI

## TODO: Check that the parameterised phase allows you to change the reflexivity of your MZI


### 3. Universal circuits

An operation on the modes of our circuit can also be expressed as a unitary.

For three modes, the unitary $U=\begin{pmatrix}
a_{1,1} & a_{1,2} & a_{1,3}\\
a_{2,1} & a_{2,2} & a_{2,3} \\ 
a_{3,1} & a_{3,2} & a_{3,3}
\end{pmatrix}$ performs the following operation on the Fock state basis:

$$\begin{array}{rcl}
|1,0,0\rangle &  \mapsto&  a_{1,1}|1,0,0\rangle + a_{1,2}|0,1,0\rangle + a_{1,3}|0,0,1\rangle\\
|0,1,0\rangle &  \mapsto&  a_{2,1}|1,0,0\rangle + a_{2,2}|0,1,0\rangle + a_{2,3}|0,0,1\rangle\\
|0,0,1\rangle &  \mapsto&  a_{3,1}|1,0,0\rangle + a_{3,2}|0,1,0\rangle + a_{3,3}|0,0,1\rangle
\end{array}$$

In 1994, [Reck et al](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.73.58) showed that any $U$ on these modes can be implemented as an LO-circuit.

This decomposition can easily be done in Perceval using beam splitters and phase shifters. 

In [ ]:
## Generate a random unitary on n modes
n=3
U=pcvl.Matrix.random_unitary(n)

## Decompose the unitary according to Reck et al
circuit_u=pcvl.Circuit.decomposition(U,BS(theta=pcvl.P('theta'),phi_tr=pcvl.P('phi')),phase_shifter_fn=PS)

pcvl.pdisplay(circuit_u)

In [ ]:
print('The error between the two unitaries is',np.linalg.norm(U-circuit_u.compute_unitary()))

In [ ]:
## TODO: decompose your unitary with only phase shifters and balanced beamsplitters (MZIs).


In [ ]:
## TODO: check the norm of the difference to be sure it has worked well 


### 4. Black box

To improve readibility, the circuit can be constructed in multiple steps which are then combined as black boxes. This will also help when we'll need generic operations.


In [ ]:
pre_MZI = (pcvl.Circuit(4, name='Bell State Preparation')
           .add(0, BS())
           .add(2, BS())
           .add(1, PERM([1, 0])))

upper_MZI = (pcvl.Circuit(2, name='upper MZI')
             .add(0, PS(phi=pcvl.P('phi_0')))
             .add(0, BS())
             .add(0, PS(phi=pcvl.P('phi_2')))
             .add(0, BS()))

lower_MZI = (pcvl.Circuit(2, name='lower MZI')
             .add(0, PS(phi=pcvl.P('phi_1')))
             .add(0, BS())
             .add(0, PS(phi=pcvl.P('phi_3')))
             .add(0, BS()))

chip = (pcvl.Circuit(4)
              .add(0, pre_MZI)
              .add(0, upper_MZI, merge=False) # merge is False to show boxes, True shows sub-circuits
              .add(2, lower_MZI, merge=False))

pcvl.pdisplay(chip)

In [ ]:
## You can still display the inside of black boxes with:
pcvl.pdisplay(chip, recursive=True)

## III. Simulation

Up until this point, we have focused on creating circuits.
It's time to learn how to sample from them or describe their output distribution.

### 1. Computing probabilities

We will take the [Hong-Ou-Mandel](https://en.wikipedia.org/wiki/Hong%E2%80%93Ou%E2%80%93Mandel_effect) experiment as an example.

Making two indistinguishable photons, one in each mode, enter one balanced beamsplitter $BS=\frac{1}{\sqrt{2}} \left[\begin{matrix}1 & 1\\1& -1\end{matrix}\right]$, we expect the outcome to be bunched:

$$|1,1\rangle \mapsto \frac{|2,0\rangle - |0,2\rangle}{\sqrt{2}}  $$

We will show how to verify this in the next steps using the Naive backend to recover the full probability distribution.

In [ ]:
## TODO: build the circuit with the convention above


In [ ]:
# Syntax to compute the amplitudes
backend = pcvl.BackendFactory.get_backend('Naive') 

simulator = backend(circuit)
print(simulator.probampli(pcvl.BasicState([1,1]), pcvl.BasicState([2,0]))) # note that it's the amplitude! 
print(simulator.probampli(pcvl.BasicState([1,1]), pcvl.BasicState([0,2])))
print(simulator.prob(pcvl.BasicState([1,1]), pcvl.BasicState([2,0]))) # note that it's the probability!
print(simulator.prob(pcvl.BasicState([1,1]), pcvl.BasicState([0,2])))

## We can also use the Analyser module to compute a table of probabilities
## The Analyzer works with a Processor. A Processor simulates a photon source plugged into a circuit.
## The main syntax is:
## >>> p = pcvl.Processor(backend_name, circuit, source)
p = pcvl.Processor('Naive', BS())
analyzer = Analyzer(p, [pcvl.BasicState([1,1])], '*')
pcvl.pdisplay(analyzer)

In [ ]:
## TODO:  Choose a random unitary 3x3 U, and output the  probablity table given the input |1,1,0>.


### 2. Sampling

Although it's crucial to compute the output distribution, it's not what we can expect from a photonic chip. Indeed, realistically, we only can obtain a single sample from the distribution each time we run the circuit. This can be done using the backend SLOS.



In [ ]:
p = pcvl.Processor('SLOS', BS())
p.with_input(pcvl.BasicState([1,1]))

# The sampler holds 'probs', 'sample_count' and 'samples' calls. You can use the one that fits your needs!
sampler = Sampler(p)  

sample_count = sampler.sample_count(1000)
print(sample_count['results'])

In [ ]:
## TODO: implement the code to sample from the 3x3 Unitary of earlier

## Question: how many states do we have for 3 modes and 2 photons?
## Answer: 

## Question: how many states do we have for m modes and n photons? 
## Answer: 

### 3. Variational algorithm

In variational algorithms, the samples from a quantum circuit allow us to approximate an expectation value, which is then used to determine the value of a loss function. This loss function is chosen such that minimising it yields a solution to a given problem. By changing the values of the parameters in our quantum circuit, we can search for this minimum.

We won't go into the details of variational algorithms. However, it may be useful to see how to perform an optimisation with Perceval.

We will use the library [scipy.optimise](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize).

The following code solves the problem of finding an LO-circuit which, given an input Fock state $|1,1,1,1\rangle$, maximizes the probability of outputting $|4,0,0,0\rangle$.

The solution below works for an arbitrary $n$.

In [ ]:
from scipy import optimize

## Data
n=4
input=pcvl.BasicState([1]*n)
output_to_max=pcvl.BasicState([n]+[0]*(n-1))
backend = pcvl.BackendFactory.get_backend('Naive') 

## TODO: implement a generic circuit of size n with parameters. 
## Code the loss function to maximise the desired output. 
## Launch the optimisation procedure. 
## Output the probability and circuit obtained.


## IV. Remote computation on a real QPU

Perceval can also connect to Quandela's photonic quantum processor on the cloud.

In [ ]:
import time
from tqdm.notebook import tqdm

First, define your circuit and input state using Perceval, as usual.

In [ ]:
input_state = pcvl.BasicState([1, 1])

c = pcvl.Circuit(2)
c.add(0, pcvl.BS())
c.add(0, pcvl.PS(phi = np.pi/4))
c.add(0, pcvl.BS())

pcvl.pdisplay(c)

Now, create an account on [cloud.quandela.com](https://cloud.quandela.com/) and login to see which QPU and simulators are available, as well as their specifications. Once you have chosen, all you have to do is to copy the machine’s name. You can now define a RemoteProcessor using the name of the machine and your token. Before using your token, don’t forget to give it the right to be used on the machine you want.

In [ ]:
# Use your key here to let the system know who you are
token_qcloud = 'YOUR_API_KEY'
remote_simulator = pcvl.RemoteProcessor('sim:ascella', token_qcloud)

You can now remotely access the simulator specs:

In [ ]:
specs = remote_simulator.specs
pcvl.pdisplay(specs['specific_circuit'])

In [ ]:
print(specs['constraints'])
print(specs['parameters'])

Now we have to specify which parameters we want to give to compute. For specific parameters, we have to use a special `set_parameter` function (or `set_parameters`).

In [ ]:
remote_simulator = pcvl.RemoteProcessor('sim:ascella', token_qcloud)
remote_simulator.set_circuit(c)
remote_simulator.with_input(input_state)
remote_simulator.min_detected_photons_filter(1)

remote_simulator.set_parameters({
    'HOM': .9461,
    'transmittance': .0765,
    'g2': .0086
})

nsample = 10000

We can now use the `Sampler` with our `RemoteProcessor`.

In [ ]:
sampler = Sampler(remote_simulator)

remote_job = sampler.sample_count
remote_job.name = 'My CiQ sampling job' # All jobs created by this sampler instance will have this custom name on the cloud
remote_job.execute_async(nsample)

The job has now been sent to a distant computer. As it is an async computation, we can do other things locally before the results arrive. In our case, we will just wait for the end of the computation. If you go to the cloud website again, you can see the job and its completion status.

In [ ]:
previous_prog = 0
with tqdm(total=1, bar_format='{desc}{percentage:3.0f}%|{bar}|') as tq:
    tq.set_description(f'Get {nsample} samples from {remote_simulator.name}')
    while not remote_job.is_complete:
        tq.update(remote_job.status.progress/100-previous_prog)
        previous_prog = remote_job.status.progress/100
        time.sleep(1)
    tq.update(1-previous_prog)
    tq.close()

print(f'Job status = {remote_job.status()}')

Once the previous cell has stopped, the job is finished. We can now retrieve the results for some computation. Here, the computation should be relatively fast (unless the simulator is unavailable or there are many requests on it), so we can use the job object we created before.

In [ ]:
results = remote_job.get_results()
print(results['results'])

You can also run the same sampling on the QPU:

In [ ]:
remote_qpu = pcvl.RemoteProcessor('qpu:ascella', token_qcloud)
remote_qpu.set_circuit(c)
remote_qpu.with_input(input_state)
remote_qpu.min_detected_photons_filter(1)

nsample = 10000

sampler_on_qpu = Sampler(remote_qpu)

remote_job = sampler_on_qpu.sample_count
remote_job.name = 'QPU CiQ sampling job'
remote_job.execute_async(nsample)

In [ ]:
previous_prog = 0
with tqdm(total=1, bar_format='{desc}{percentage:3.0f}%|{bar}|') as tq:
    tq.set_description(f'Get {nsample} samples from {remote_qpu.name}')
    while not remote_job.is_complete:
        tq.update(remote_job.status.progress/100-previous_prog)
        previous_prog = remote_job.status.progress/100
        time.sleep(1)
    tq.update(1-previous_prog)
    tq.close()

print(f'Job status = {remote_job.status()}')

In [ ]:
results = remote_job.get_results()
print(results['results'])
## may print >10k counts if your job takes less than 1 second
## because collection rate is 1Hz, but photon clock rate is >>1Hz